<a href="https://colab.research.google.com/github/TanyaPaquet/mypackage/blob/master/zindi_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Import some necessary libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
#Load the provided training and test data into dataframes
train = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/zindi/training.csv' )
test = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/zindi/test.csv' )

#Check the dataframe sizes, type assignments, and missing values
print('Train info:')
train.info()
print('\nTest info:')
test.info()

Train info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95662 entries, 0 to 95661
Data columns (total 16 columns):
TransactionId           95662 non-null object
BatchId                 95662 non-null object
AccountId               95662 non-null object
SubscriptionId          95662 non-null object
CustomerId              95662 non-null object
CurrencyCode            95662 non-null object
CountryCode             95662 non-null int64
ProviderId              95662 non-null object
ProductId               95662 non-null object
ProductCategory         95662 non-null object
ChannelId               95662 non-null object
Amount                  95662 non-null float64
Value                   95662 non-null int64
TransactionStartTime    95662 non-null object
PricingStrategy         95662 non-null int64
FraudResult             95662 non-null int64
dtypes: float64(1), int64(4), object(11)
memory usage: 11.7+ MB

Test info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45019 entries, 0 to 

In [4]:
train.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [5]:
for col in train.columns:
  print(col, len(train[col].unique()))

TransactionId 95662
BatchId 94809
AccountId 3633
SubscriptionId 3627
CustomerId 3742
CurrencyCode 1
CountryCode 1
ProviderId 6
ProductId 23
ProductCategory 9
ChannelId 4
Amount 1676
Value 1517
TransactionStartTime 94556
PricingStrategy 4
FraudResult 2


In [6]:
num_fraud = len(train[train['FraudResult'] == 1])

print('Number of fraudulent transactions: ', num_fraud)
print('Fraction of fraudulent transactions: ', num_fraud/(len(train)))

Number of fraudulent transactions:  193
Fraction of fraudulent transactions:  0.00201752001839811


In [0]:
combined = pd.concat([train.drop('FraudResult', axis=1), test])
work_df = combined.drop(['TransactionId', 'BatchId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode'], axis=1)

In [0]:
work_df['TransactionStartTime'] = pd.to_datetime(work_df['TransactionStartTime'])
work_df['PricingStrategy'] = work_df['PricingStrategy'].astype(object)

In [0]:
work_df['day'] = work_df['TransactionStartTime'].dt.day
work_df['hour'] = work_df['TransactionStartTime'].dt.hour
work_df['week_day'] = work_df['TransactionStartTime'].dt.dayofweek

In [10]:
work_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140681 entries, 0 to 45018
Data columns (total 12 columns):
AccountId               140681 non-null object
ProviderId              140681 non-null object
ProductId               140681 non-null object
ProductCategory         140681 non-null object
ChannelId               140681 non-null object
Amount                  140681 non-null float64
Value                   140681 non-null int64
TransactionStartTime    140681 non-null datetime64[ns, UTC]
PricingStrategy         140681 non-null object
day                     140681 non-null int64
hour                    140681 non-null int64
week_day                140681 non-null int64
dtypes: datetime64[ns, UTC](1), float64(1), int64(4), object(6)
memory usage: 14.0+ MB


In [0]:
def tod(t):
    if t >= 0 and t < 3:
        return 'early_morning'
    elif t >= 3 and t < 6:
        return 'morning'
    elif t >= 6 and t < 9:
        return 'late_morning'
    elif t >= 9 and t < 12:
        return 'early_afternoon'
    elif t >= 12 and t < 15:
        return 'afternoon'
    elif t >= 15 and t < 18:
        return 'late_afternoon'
    elif t >= 18 and t < 21:
        return 'early_evening'
    elif t >= 21:
        return 'lave_evening'
    return 'other'
  

work_df['time_day'] = work_df['hour'].apply(lambda row: tod(row))

In [12]:
fraud = pd.concat([work_df.iloc[:len(train)], train['FraudResult']], axis=1)

fraud = fraud[fraud['FraudResult'] == 1]
fraud.describe()

,Amount,Value,day,hour,week_day,FraudResult
count,1.930000e+02,1.930000e+02,193.000000,193.000000,193.000000,193.0
mean,1.535272e+06,1.561820e+06,14.181347,13.341969,2.725389,1.0
std,2.100525e+06,2.081227e+06,9.276609,5.073127,1.860283,0.0
min,-9.000000e+05,5.000000e+02,1.000000,0.000000,0.000000,1.0
25%,5.000000e+05,5.000000e+05,6.000000,10.000000,1.000000,1.0
50%,6.000000e+05,6.500000e+05,12.000000,13.000000,3.000000,1.0
75%,2.000000e+06,2.000000e+06,22.000000,17.000000,4.000000,1.0
max,9.880000e+06,9.880000e+06,31.000000,22.000000,6.000000,1.0


In [0]:
fraud['diff'] = np.abs(fraud['Amount']) - np.abs(fraud['Value'])

In [14]:
for col in fraud.columns:
  print(col, len(fraud[col].unique()))

AccountId 52
ProviderId 5
ProductId 9
ProductCategory 4
ChannelId 3
Amount 61
Value 61
TransactionStartTime 193
PricingStrategy 4
day 30
hour 20
week_day 7
time_day 8
FraudResult 1
diff 6


Thus: date and time will not offer much distinction

In [15]:
work_df.head()

,AccountId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,day,hour,week_day,time_day
0,AccountId_3957,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,15,2,3,early_morning
1,AccountId_4841,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,15,2,3,early_morning
2,AccountId_4229,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,15,2,3,early_morning
3,AccountId_648,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,15,3,3,morning
4,AccountId_4841,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,15,3,3,morning


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import f1_score, confusion_matrix, classification_report, roc_auc_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [17]:
dummies = work_df[['ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Value', 'PricingStrategy']]
dummies = pd.get_dummies(dummies, drop_first=True)

# dummies = work_df[['AccountId', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Value', 'PricingStrategy']]
# sc = MinMaxScaler()
# dummies_sc = sc.fit_transform(dummies)

dummies.shape

(140681, 48)

In [0]:
train_dum = dummies.iloc[: len(train)]
test_dum = dummies.iloc[len(train):]

In [0]:
#Define model building function
def build_model(train_X, test_X, train_y, test_y, model):
    """Split data into training and test sets, fit the specified model, make predictions, and calculate the log loss and confusion matrix.
    
    Args:
        X (array or DataFrame): independent variables
        y (array or DataFrame): dependent variable
        model (model): sklearn model and parameters

    Returns:
        train_pred (array): predictions from training data 
        test_pred (array): predictions from test data
        train_loss (float): log loss on training data
        test_loss (float): log loss on test data
        test_matrix (array): confusion matrix on test data
        
    """
    
    model.fit(train_X, train_y)

    train_pred = model.predict(train_X)
    test_pred = model.predict(test_X)
    
    train_proba = model.predict_proba(train_X)
    test_proba = model.predict_proba(test_X)
    
    
    return train_pred, test_pred, train_proba, test_proba

##  RF train_test_split on subset of 0.3

In [359]:
train_fraud = pd.concat([train_dum, train['FraudResult']], axis=1)

sub_train = pd.concat([train_fraud[train_fraud['FraudResult'] == 1].sample(frac=0.3, random_state=11), train_fraud[train_fraud['FraudResult'] == 0].sample(frac=0.3, random_state=11)])

sub_train.shape

(28699, 49)

In [360]:
X_train, X_test, y_train, y_test = train_test_split(sub_train.drop('FraudResult', axis=1), sub_train['FraudResult'], random_state=11)

print(X_train.shape)
print(X_test.shape)

(21524, 48)
(7175, 48)


In [0]:
cm = RandomForestClassifier(n_estimators=50, max_depth=3, max_features='auto', bootstrap=True, n_jobs=-1, random_state=11)

pred_train, pred_test, proba_train, proba_test = build_model(X_train, X_test, y_train, y_test, cm)


In [362]:
print('train f1: ', f1_score(y_train, pred_train))
print('test f1: ', f1_score(y_test, pred_test))
print('\nConfusion matrix:')
print(confusion_matrix(y_test, pred_test))

train f1:  0.0
test f1:  0.0

Confusion matrix:
[[7159    0]
 [  16    0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [363]:
frac = [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for f in frac:
    y_train_th = np.where(proba_train[:,1] > f, 1, 0)
    y_test_th = np.where(proba_test[:,1] > f, 1, 0)
    print('\nFraction: ', f)
    print('train f1: ', f1_score(y_train, y_train_th))
    print('test f1: ', f1_score(y_test, y_test_th))
    print('Confusion matrix:')
    print(confusion_matrix(y_test, y_test_th))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)



Fraction:  0.005
train f1:  0.07466666666666666
test f1:  0.07462686567164178
Confusion matrix:
[[6788  371]
 [   1   15]]

Fraction:  0.01
train f1:  0.21761658031088082
test f1:  0.23255813953488375
Confusion matrix:
[[7061   98]
 [   1   15]]

Fraction:  0.05
train f1:  0.8297872340425532
test f1:  0.9032258064516129
Confusion matrix:
[[7158    1]
 [   2   14]]

Fraction:  0.1
train f1:  0.8297872340425532
test f1:  0.9032258064516129
Confusion matrix:
[[7158    1]
 [   2   14]]

Fraction:  0.15
train f1:  0.8297872340425532
test f1:  0.9032258064516129
Confusion matrix:
[[7158    1]
 [   2   14]]

Fraction:  0.2
train f1:  0.6756756756756757
test f1:  0.64
Confusion matrix:
[[7158    1]
 [   8    8]]

Fraction:  0.25
train f1:  0.36363636363636365
test f1:  0.2222222222222222
Confusion matrix:
[[7159    0]
 [  14    2]]

Fraction:  0.3
train f1:  0.046511627906976744
test f1:  0.0
Confusion matrix:
[[7159    0]
 [  16    0]]

Fraction:  0.35
train f1:  0.0
test f1:  0.0
Confusion 

##  RF train_test_split on all 

In [376]:
train_fraud = pd.concat([train_dum, train['FraudResult']], axis=1)

train_fraud.shape

(95662, 49)

In [377]:
X_train, X_test, y_train, y_test = train_test_split(train_fraud.drop('FraudResult', axis=1), train_fraud['FraudResult'], random_state=11)

print(X_train.shape)
print(X_test.shape)

(71746, 48)
(23916, 48)


In [0]:
cm = RandomForestClassifier(n_estimators=50, max_depth=3, max_features='auto', bootstrap=True, n_jobs=-1, random_state=11)

pred_train, pred_test, proba_train, proba_test = build_model(X_train, X_test, y_train, y_test, cm)


In [379]:
print('train f1: ', f1_score(y_train, pred_train))
print('test f1: ', f1_score(y_test, pred_test))
print('\nConfusion matrix:')
print(confusion_matrix(y_test, pred_test))

train f1:  0.0
test f1:  0.0

Confusion matrix:
[[23866     0]
 [   50     0]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [380]:
frac = [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for f in frac:
    y_train_th = np.where(proba_train[:,1] > f, 1, 0)
    y_test_th = np.where(proba_test[:,1] > f, 1, 0)
    print('\nFraction: ', f)
    print('train f1: ', f1_score(y_train, y_train_th))
    print('test f1: ', f1_score(y_test, y_test_th))
    print('Confusion matrix:')
    print(confusion_matrix(y_test, y_test_th))


Fraction:  0.005
train f1:  0.030017152658662095
test f1:  0.03096774193548387
Confusion matrix:
[[20864  3002]
 [    2    48]]

Fraction:  0.01
train f1:  0.22012578616352202
test f1:  0.21967963386727687
Confusion matrix:
[[23527   339]
 [    2    48]]

Fraction:  0.05
train f1:  0.7962382445141066
test f1:  0.7796610169491526
Confusion matrix:
[[23844    22]
 [    4    46]]

Fraction:  0.1
train f1:  0.7949526813880126
test f1:  0.8288288288288288
Confusion matrix:
[[23851    15]
 [    4    46]]

Fraction:  0.15
train f1:  0.7999999999999999
test f1:  0.851851851851852
Confusion matrix:
[[23854    12]
 [    4    46]]

Fraction:  0.2
train f1:  0.7349823321554771
test f1:  0.7878787878787878
Confusion matrix:
[[23856    10]
 [   11    39]]

Fraction:  0.25
train f1:  0.3176470588235294
test f1:  0.3050847457627119
Confusion matrix:
[[23866     0]
 [   41     9]]

Fraction:  0.3
train f1:  0.027586206896551724
test f1:  0.0392156862745098
Confusion matrix:
[[23866     0]
 [   49     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


##  RF using all test data


In [0]:
cm = RandomForestClassifier(n_estimators=50, max_depth=3, max_features='auto', bootstrap=True, n_jobs=-1, random_state=11)

cm.fit(train_dum, train['FraudResult'])

pred_train = cm.predict(train_dum)
pred_test = cm.predict(test_dum)
    
proba_train = cm.predict_proba(train_dum)
proba_test = cm.predict_proba(test_dum)

In [382]:
frac = [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for f in frac:
    y_train_th = np.where(proba_train[:,1] > f, 1, 0)
    print('\nFraction: ', f)
    print('train f1: ', f1_score(train['FraudResult'], y_train_th))


Fraction:  0.005
train f1:  0.030939586386581987

Fraction:  0.01
train f1:  0.22248520710059172

Fraction:  0.05
train f1:  0.8113207547169811

Fraction:  0.1
train f1:  0.8113207547169811

Fraction:  0.15
train f1:  0.8113207547169811

Fraction:  0.2
train f1:  0.8105515587529976

Fraction:  0.25
train f1:  0.4030418250950569

Fraction:  0.3
train f1:  0.030612244897959183

Fraction:  0.35
train f1:  0.0

Fraction:  0.4
train f1:  0.0

Fraction:  0.45
train f1:  0.0

Fraction:  0.5
train f1:  0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [387]:
y_thresh = np.where(proba_test[:,1] > 0.15, 1, 0)
y_thresh

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
results = test[['TransactionId']]

In [389]:
results['FraudResult'] = y_thresh

results.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,TransactionId,FraudResult
0,TransactionId_50600,0
1,TransactionId_95109,0
2,TransactionId_47357,0
3,TransactionId_28185,0
4,TransactionId_22140,0


In [0]:
# results.to_csv('/content/gdrive/My Drive/Colab Notebooks/zindi/RF_50_3_auto_0-15.csv', index=False)

## GB train_test_split on all 

In [326]:
train_fraud = pd.concat([train_dum, train['FraudResult']], axis=1)

train_fraud.shape

(95662, 49)

In [327]:
X_train, X_test, y_train, y_test = train_test_split(train_fraud.drop('FraudResult', axis=1), train_fraud['FraudResult'], random_state=11)

print(X_train.shape)
print(X_test.shape)

(71746, 48)
(23916, 48)


In [0]:
cm = GradientBoostingClassifier(loss='exponential', n_estimators=50, max_depth=3, random_state=11, max_features=None)

pred_train, pred_test, proba_train, proba_test = build_model(X_train, X_test, y_train, y_test, cm)


In [335]:
print('train f1: ', f1_score(y_train, pred_train))
print('test f1: ', f1_score(y_test, pred_test))
print('\nConfusion matrix:')
print(confusion_matrix(y_test, pred_test))

train f1:  0.8705882352941178
test f1:  0.8817204301075269

Confusion matrix:
[[23864     2]
 [    9    41]]


In [336]:
frac = [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for f in frac:
    y_train_th = np.where(proba_train[:,1] > f, 1, 0)
    y_test_th = np.where(proba_test[:,1] > f, 1, 0)
    print('\nFraction: ', f)
    print('train f1: ', f1_score(y_train, y_train_th))
    print('test f1: ', f1_score(y_test, y_test_th))
    print('Confusion matrix:')
    print(confusion_matrix(y_test, y_test_th))


Fraction:  0.005
train f1:  0.5371428571428571
test f1:  0.5444444444444444
Confusion matrix:
[[23785    81]
 [    1    49]]

Fraction:  0.01
train f1:  0.5743380855397149
test f1:  0.5664739884393063
Confusion matrix:
[[23792    74]
 [    1    49]]

Fraction:  0.05
train f1:  0.6731234866828089
test f1:  0.6712328767123288
Confusion matrix:
[[23819    47]
 [    1    49]]

Fraction:  0.1
train f1:  0.7061855670103094
test f1:  0.7164179104477612
Confusion matrix:
[[23830    36]
 [    2    48]]

Fraction:  0.15
train f1:  0.8167202572347266
test f1:  0.851851851851852
Confusion matrix:
[[23854    12]
 [    4    46]]

Fraction:  0.2
train f1:  0.8723404255319149
test f1:  0.8846153846153846
Confusion matrix:
[[23858     8]
 [    4    46]]

Fraction:  0.25
train f1:  0.8723404255319149
test f1:  0.8932038834951457
Confusion matrix:
[[23859     7]
 [    4    46]]

Fraction:  0.3
train f1:  0.8723404255319149
test f1:  0.8932038834951457
Confusion matrix:
[[23859     7]
 [    4    46]]

Fr

##  GB using all test data


In [0]:
cm = GradientBoostingClassifier(loss='exponential', n_estimators=50, max_depth=3, random_state=11, max_features=None)

cm.fit(train_dum, train['FraudResult'])

pred_train = cm.predict(train_dum)
pred_test = cm.predict(test_dum)
    
proba_train = cm.predict_proba(train_dum)
proba_test = cm.predict_proba(test_dum)

In [338]:
frac = [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for f in frac:
    y_train_th = np.where(proba_train[:,1] > f, 1, 0)
    print('\nFraction: ', f)
    print('train f1: ', f1_score(train['FraudResult'], y_train_th))


Fraction:  0.005
train f1:  0.5654761904761905

Fraction:  0.01
train f1:  0.6308724832214765

Fraction:  0.05
train f1:  0.7269155206286837

Fraction:  0.1
train f1:  0.8169014084507042

Fraction:  0.15
train f1:  0.8297362110311751

Fraction:  0.2
train f1:  0.8802083333333334

Fraction:  0.25
train f1:  0.8802083333333334

Fraction:  0.3
train f1:  0.8802083333333334

Fraction:  0.35
train f1:  0.877284595300261

Fraction:  0.4
train f1:  0.8924731182795699

Fraction:  0.45
train f1:  0.8983050847457626

Fraction:  0.5
train f1:  0.8977272727272727


In [339]:
y_thresh = np.where(proba_test[:,1] > 0.2, 1, 0)
y_thresh

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
results = test[['TransactionId']]

In [341]:
results['FraudResult'] = y_thresh

results.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,TransactionId,FraudResult
0,TransactionId_50600,0
1,TransactionId_95109,0
2,TransactionId_47357,0
3,TransactionId_28185,0
4,TransactionId_22140,0


In [0]:
# results.to_csv('/content/gdrive/My Drive/Colab Notebooks/zindi/GB_exp_50_3_none_0-2.csv', index=False)